In [ ]:
import array
import random
import numpy as np
import pandas as pd
import glob
import os.path
import sys
from datetime import datetime

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, mean_squared_error

from metric_learn import LMNN, NCA, LFDA
#from metric_learn import ITML_Supervised, SDML_Supervised, LSML_Supervised, RCA_Supervised

from deap import algorithms
from deap import base
from deap import benchmarks
from deap import cma
from deap import creator
from deap import tools

from concurrent.futures import ThreadPoolExecutor 

In [ ]:
data = pd.read_csv('datasets/breast-cancer-wisconsin.csv', sep=',', skiprows=1, header=0)

y = data['class']
X = data.drop(['class'], axis=1).values

# Problem size
N=X.shape[1]

X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=47)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=0, copy=False)
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1, n_jobs=-1)
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

In [ ]:
class ESMetric():
    def __init__(self, L, N, isFull=False):
        self.params = {}
        self.N = N
        if N == len(L):
            self.L = np.diag(L)
        elif N**2 == len(L):
            self.L = np.reshape(L, (N, N))
        else:
            raise Error('Invalid size of N')

    def transform(self, X):
        return X.dot(self.transformer().T)
        
    def transformer(self):
        return self.L

In [ ]:
def knnEvaluation(ind):
    es = ESMetric(ind, N)
    X_train_trans = es.transform(X_train)
    X_test_trans = es.transform(X_test)
    knn = KNeighborsClassifier(n_neighbors=8, n_jobs=-1)
    knn.fit(X_train_trans, y_train)
    score = knn.score(X_test_trans, y_test)
#     print(score)
    return [score]
    return [score - mean_squared_error(ind, np.ones(N))]
    return [score - np.sum(np.absolute(ind))]

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("evaluate", knnEvaluation)
toolbox.register("map", ThreadPoolExecutor(max_workers=None).map)

In [ ]:
# The cma module uses the numpy random number generator
np.random.seed(128)

# The CMA-ES algorithm takes a population of one individual as argument
# The centroid is set to a vector of 5.0 see http://www.lri.fr/~hansen/cmaes_inmatlab.html
# for more details about the rastrigin and other tests for CMA-ES
strategy = cma.Strategy(centroid=[1.0]*N*N, sigma=5.0) # lambda_=20*N
toolbox.register("generate", strategy.generate, creator.Individual)
toolbox.register("update", strategy.update)

hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
# stats.register("std", numpy.std)
# stats.register("min", numpy.min)
stats.register("max", np.max)
#logger = tools.EvolutionLogger(stats.functions.keys())

In [ ]:
# The CMA-ES algorithm converge with good probability with those settings
pop, logbook = algorithms.eaGenerateUpdate(toolbox, ngen=250, stats=stats, halloffame=hof)

In [ ]:
# print "Best individual is %s, %s" % (hof[0], hof[0].fitness.values)
hof[0].fitness.values[0]
hof[0], knnEvaluation(np.abs(hof[0]))
knnEvaluation(hof[0]), hof[0]

In [ ]:
pop

In [ ]:
logbook